### Introduction

Video games are a popular form of entertainment, but not everyone plays in the same way. Some players spend many hours exploring and interacting with the game, while others play only briefly. Understanding what influences how long people play can help researchers learn about player behavior and engagement.

At the University of British Columbia, a research team is studying this using a Minecraft server. On the server, players can explore freely and interact with the environment, and the researchers record their actions. Along with playtime, the team collects information about each player, such as their experience level, subscription status, age, and gender.

The goal of this project is to see whether these characteristics can help **predict how long a player will spend in the game**. Specifically, we ask: **Can we estimate the total number of hours a user will play (played_hours) based on their experience, subscription status, gender, and age?** By examining these relationships, we hope to identify factors that are linked to higher or lower engagement.

### Dataset 1 (`players.csv`)

This dataset contains information about **196 individuals**, each representing a player profile. It includes **9 variables** capturing demographic information, subscription status, and engagement metrics. Below is a detailed description of each variable:

| Variable Name     | Dtype    | Description              | Summary Statistic |
|-------------------|----------|---------------------------|-------------------|
| experience        | object   | Player expertise level    | – |
| subscribe         | bool     | Subscription status       | – |
| hashedEmail       | object   | Encrypted email ID        | – |
| played_hours      | float64  | Total playtime (hours)    | refer to summary_stats |
| name              | object   | Participant name          | – |
| gender            | object   | Gender                    | – |
| age               | int64    | Player age (years)        | refer to summary_stats |
| individualId      | float64  | -              | – |
| organizationName  | float64  |-              | – |


**Note:** The `individualId` and `organizationName` columns are entirely empty and provide no usable information. These should be removed during preprocessing.  These fields may not have been collected, applicable, or were lost during data processing.
 
---

### Dataset Issues and Considerations

While reviewing the dataset, several important points were noted:

- **Columns with all missing values:**  
  `individualId` and `organizationName` contain 100% missing data. These likely were placeholders or never collected.

- **Fields not useful for prediction:**  
  Columns like `name` or `hashedEmail` are identifiers and don’t provide meaningful information for modeling.

- **Categorical variables need encoding:**  
  - `experience` may follow an **ordinal scale** (Beginner → Pro).  
  - `gender` is **non-ordinal**, so it should be one-hot encoded for models like KNN.

- **Self-reported data:**  
  Fields like experience level and age may contain bias or inaccuracies, which should be considered when interpreting results.

### Relevance for KNN Regression

Some features of this dataset are especially important when using K-Nearest Neighbors (KNN) regression:  

- **Numeric features only:** KNN measures distance between points, so any categorical variables (like subscription status or gender) need to be converted to numbers.  
- **Scaling matters:** Features such as `age` and `played_hours` are on different scales. Without scaling, variables with larger ranges could dominate the distance calculations.  
- **Remove unnecessary fields:** Columns like `name` or `hashedEmail` don’t provide useful information for predicting playtime and can distort distances between players.  
- **Watch out for outliers:** Extremely high or low values, especially in age, can have a big effect on which points are considered neighbors.  
- **Uneven experience levels:** If some experience categories have very few players, KNN might struggle to find representative neighbors for them.  

In [17]:
import altair as alt
import numpy as np
import pandas as pd
from sklearn import set_config
from sklearn.model_selection import GridSearchCV, cross_validate, train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.metrics import mean_squared_error
from sklearn.compose import make_column_transformer

In [2]:
url='https://drive.google.com/uc?export=download&id=1Mw9vW0hjTJwRWx0bDXiSpYsO3gKogaPz'
player_data=pd.read_csv(url)
player_data

,experience,subscribe,hashedEmail,played_hours,name,gender,age,individualId,organizationName
0,Pro,True,f6daba428a5e19a3d47574858c13550499be23603422e6...,30.3,Morgan,Male,9,NaN,NaN
1,Veteran,True,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa9397...,3.8,Christian,Male,17,NaN,NaN
2,Veteran,False,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3...,0.0,Blake,Male,17,NaN,NaN
3,Amateur,True,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4f...,0.7,Flora,Female,21,NaN,NaN
4,Regular,True,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb...,0.1,Kylie,Male,21,NaN,NaN
...,...,...,...,...,...,...,...,...,...
191,Amateur,True,b6e9e593b9ec51c5e335457341c324c34a2239531e1890...,0.0,Bailey,Female,17,NaN,NaN
192,Veteran,False,71453e425f07d10da4fa2b349c83e73ccdf0fb3312f778...,0.3,Pascal,Male,22,NaN,NaN
193,Amateur,False,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db29...,0.0,Dylan,Prefer not to say,17,NaN,NaN
194,Amateur,False,f19e136ddde68f365afc860c725ccff54307dedd13968e...,2.3,Harlow,Male,17,NaN,NaN


In [3]:
summary_stats = player_data[["played_hours","age"]].describe()
summary_stats

,played_hours,age
count,196.000000,196.000000
mean,5.845918,21.280612
std,28.357343,9.706346
min,0.000000,8.000000
25%,0.000000,17.000000
50%,0.100000,19.000000
75%,0.600000,22.000000
max,223.100000,99.000000


In [4]:
player_relevant_data=player_data.drop(columns=['hashedEmail', 'individualId', 'organizationName', 'name'])
player_relevant_data

,experience,subscribe,played_hours,gender,age
0,Pro,True,30.3,Male,9
1,Veteran,True,3.8,Male,17
2,Veteran,False,0.0,Male,17
3,Amateur,True,0.7,Female,21
4,Regular,True,0.1,Male,21
...,...,...,...,...,...
191,Amateur,True,0.0,Female,17
192,Veteran,False,0.3,Male,22
193,Amateur,False,0.0,Prefer not to say,17
194,Amateur,False,2.3,Male,17


In [5]:
# Map subscription status to consistent labels
player_relevant_data['subscribe'] = (
    player_relevant_data['subscribe']
    .map({True: 'Subscribed',
          False: 'Not Subscribed',
          'Subscribed': 'Subscribed',
          'Not Subscribed': 'Not Subscribed'})
)
    
# Map genders to Male / Female / Other
gender_map = {
    'Male': 'Male',
    'Female': 'Female',
    'Prefer not to say': 'Other',
    'Non-binary': 'Other',
    'Agender': 'Other',
    'Two-Spirited': 'Other',
    'Other': 'Other'
}
player_relevant_data['gender'] = player_relevant_data['gender'].map(gender_map)

from sklearn.model_selection import train_test_split

features = ['experience', 'subscribe', 'gender', 'age']
target = 'played_hours'

player_training, player_testing = train_test_split(
    player_relevant_data,
    test_size=0.25,
    random_state=2024
)

X_train = player_training[features]
y_train = player_training[target]

X_test = player_testing[features]
y_test = player_testing[target]

In [6]:
bounds = player_data["played_hours"].quantile([0.9])
bounds

0.9    2.8
Name: played_hours, dtype: float64

In [7]:
# Experience facet
chart_exp = (
    alt.Chart(player_training)
    .mark_bar()
    .encode(
        x=alt.X('played_hours:Q', bin=alt.Bin(maxbins=20), title='Played Hours (hrs)'),
        y=alt.Y('count()', title='Number of Players'),
        color='experience:N'
    )
    .properties(title='Played Hours Distribution by Experience')
    .facet(column='experience:N')
)

# Subscribe facet
chart_sub = (
    alt.Chart(player_training)
    .mark_bar()
    .encode(
        x=alt.X('played_hours:Q', bin=alt.Bin(maxbins=20), title='Played Hours (hrs)'),
        y=alt.Y('count()', title='Number of Players'),
        color=alt.Color('subscribe:N').title("Subscribed")
    )
    .properties(title='Played Hours Distribution by Subscription')
    .facet(column='subscribe:N')
)

# Subscribe facet
chart_gen = (
    alt.Chart(player_training)
    .mark_bar()
    .encode(
        x=alt.X('played_hours:Q', bin=alt.Bin(maxbins=20), title='Played Hours (hrs)'),
        y=alt.Y('count()', title='Number of Players'),
        color=alt.Color('gender:N').title("gender")
    )
    .properties(title='Played Hours Distribution by Gender')
    .facet(column='gender:N')
)

#Age vs Played-Hours Scatter Plot
chart_age = (
    alt.Chart(player_training)
    .mark_point(opacity=0.5)
    .encode(
        x=alt.X('played_hours:Q', title='Played Hours (hrs)'),
        y=alt.Y('age:Q', title='Age of Players (yrs)')
    )
    .properties(
        title='Age vs Played Hours'
    )
)


chart_exp & chart_sub & chart_gen & chart_age

alt.VConcatChart(...)

In [8]:
# Experience facet (Zoomed - It was very hard to see trends from the above plots, so the plots below were created which are 'zoomed in' to the 90% quantile of data, which was found above)
chart_exp_zoom = (
    alt.Chart(player_training)
    .mark_bar(clip=True)
    .encode(
        x=alt.X('played_hours:Q',
                bin=alt.Bin(maxbins=500),
                scale=alt.Scale(domain=[0, 2.8]),   
                title='Played Hours (hrs)'),
        y=alt.Y('count()', title='Number of Players'),
        color='experience:N'
    )
    .properties(title='Played Hours Distribution by Experience (0 to 2.8 hours)')
    .facet(column='experience:N')
)

# Subscribe facet (zoomed)
chart_sub_zoom = (
    alt.Chart(player_training)
    .mark_bar(clip=True)
    .encode(
        x=alt.X('played_hours:Q',
                bin=alt.Bin(maxbins=500),
                scale=alt.Scale(domain=[0, 2.8]), 
                title='Played Hours (hrs)'),
        y=alt.Y('count()', title='Number of Players'),
        color=alt.Color('subscribe:N').title("Subscribed")
    )
    .properties(title='Played Hours Distribution by Subscription (0 to 2.8 hours)')
    .facet(column='subscribe:N')
)

chart_gen_zoom = (
    alt.Chart(player_training)
    .mark_bar(clip=True)
    .encode(
        x=alt.X('played_hours:Q',
                bin=alt.Bin(maxbins=500),
                scale=alt.Scale(domain=[0, 2.8]), 
                title='Played Hours (hrs)'),
        y=alt.Y('count()', title='Number of Players'),
        color=alt.Color('gender:N').title("gender")
    )
    .properties(title='Played Hours Distribution by Gender (0 to 2.8 hours)')
    .facet(column='gender:N')
)

#Age vs Played-Hours Scatter Plot
chart_age_zoom = (
    alt.Chart(player_training)
    .mark_point(clip=True, opacity=0.5)
    .encode(
        x=alt.X(
            'played_hours:Q',
            title='Played Hours (hrs)',
            scale=alt.Scale(domain=[0, 2.8])
        ),
        y=alt.Y('age:Q', title='Age of Players (yrs)')
    )
    .properties(
        title='Age vs Played Hours'
    )
)

chart_exp_zoom & chart_sub_zoom & chart_gen_zoom & chart_age_zoom

alt.VConcatChart(...)

In [30]:
#pre-preprocessing
player_preprocessor = make_column_transformer(
    (StandardScaler(), ["age"]),
    (OneHotEncoder(sparse_output=False),["gender", "subscribe", "experience"]),
    (OrdinalEncoder(categories=[["Beginner", "Amateur", "Regular", "Veteran", "Pro"]]), ["experience"]),
    verbose_feature_names_out=False,
    remainder="passthrough"
)

#create pipeline
player_pipe = make_pipeline(
    player_preprocessor,
    KNeighborsRegressor()
)

#finding optimal K
param_grid = {
    "kneighborsregressor__n_neighbors": range(1, 15)
}

player_gridsearch = GridSearchCV(
    estimator=player_pipe,
    param_grid=param_grid,
    cv=3,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1
)

# Fit and extract results
player_results = (
    pd.DataFrame(player_gridsearch.fit(X_train, y_train).cv_results_)
)

# Best K and its RMSE
player_best_K = player_gridsearch.best_params_
player_best_RMSPE = -player_gridsearch.best_score_

player_best_K, player_best_RMSPE

({'kneighborsregressor__n_neighbors': 9}, np.float64(22.464377891346732))

In [31]:
player_results=player_results.assign(RMSPE= -player_results["mean_test_score"])

In [32]:
#Best K graph
Optimal_K_Chart=alt.Chart(player_results).mark_line().encode(
    x=alt.X('param_kneighborsregressor__n_neighbors', title='K Value'),
    y=alt.Y('RMSPE', title='Root Mean Squared Prediction Error'))
Optimal_K_Chart

alt.Chart(...)

In [22]:
player_gridsearch.best_params_

{'kneighborsregressor__n_neighbors': 38}

In [33]:
# FINAL MODEL TESTING USING K = 4

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

X = player_relevant_data[['experience', 'subscribe', 'gender', 'age']]
y = player_relevant_data['played_hours']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=123
)

final_knn_model = make_pipeline(
    player_preprocessor,
    KNeighborsRegressor(n_neighbors=4)
)

final_knn_model.fit(X_train, y_train)

y_pred = final_knn_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

rmse

np.float64(39.850489724092476)

In [34]:
final_knn_model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('standardscaler',
                                                  StandardScaler(), ['age']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['gender', 'subscribe',
                                                   'experience']),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(categories=[['Beginner',
                                                                              'Amateur',
                                                                              'Regular',
                                                                              'Veteran',
                                                                              'Pro']]),
                                                  ['experience'])],
                                   verbose_feature_names_out=False)),
                ('kneighborsregressor', KNeighborsRegressor(n_neighbors=4))])

In [45]:
#Predicted vs Actual Plot
predicted_vs_actual = pd.DataFrame({
    "actual": y_test,
    "predicted": y_pred
})

predicted_vs_actual_chart=alt.Chart(predicted_vs_actual).mark_circle(opacity=0.5).encode(
    x=alt.X("actual",title='Actual Time Played (hours)'),
    y=alt.Y("predicted", title="Predicted Time Played (hours)")
).properties(
    title="Predicted vs Actual",
    width=300,
    height=300
)
line=alt.Chart(predicted_vs_actual).mark_line(color="black", opacity=.2).encode(
    x="actual",
    y="actual"   #y = x
)
predicted_vs_actual_chart+line

alt.LayerChart(...)

In [38]:
import pandas as pd

new_player = pd.DataFrame([{
    "experience": "Regular",
    "subscribe": "Subscribed",
    "gender": "Female",
    "age": 20
}])

predicted_hours = final_knn_model.predict(new_player)
predicted_hours

array([0.275])

In [36]:
new_player_2 = pd.DataFrame([{
    "experience": "Veteran",
    "subscribe": "Not Subscribed",
    "gender": "Male",
    "age": 80
}])

predicted_hours_2 = final_knn_model.predict(new_player_2)
predicted_hours_2

array([0.075])